### Importing libraries ###

In [ ]:
import torchtext
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import re
from argparse import Namespace
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import sentencepiece as spm
import re
import torchtext.vocab as tv
from torchtext.vocab import GloVe;
import numpy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import nltk
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


In [18]:

nltk.download('stopwords')
nltk.download('punkt_tab')

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/charlessanthakumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/charlessanthakumar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charlessanthakumar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/charlessanthakumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/charlessanthakumar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:

global_vectors = GloVe(name='6B', dim=300);

In [6]:

args = Namespace(
data_directory = "../data/balanced_dataset.csv",
seed = 1300,
max_sentence_length = 100,
lower = True,
start_token = "<sos>",
end_token = "<eos>",
unk_token = "<unk>",
pad_token = "<pad>",
vocab_size = 0,
embedding_dim = 300,
device = "cuda" if torch.cuda.is_available() else "cpu",
);

## DATA preprocessing ##
Done by Cynthia, Houmam, Abderraouf

In [7]:
toxic_frame = pd.read_csv(args.data_directory);
toxic_frame = toxic_frame.dropna()

toxic_frame

,id,comment_text,target
0,5283379,actually its a pretty important symbol of our ...,0.0
1,5775176,trump and truth are not good friends this man...,1.0
2,6012321,so must trump given how many rats have been fi...,1.0
3,6210499,harris scored 2pts in 34 minutes pathetic harr...,1.0
4,219580848446a719,try harder plz lolz a spam filter hahaha the...,1.0
...,...,...,...
9995,6085768,what this group of liberals accomplished was t...,0.0
9996,325286,start making cash right now get more time with...,0.0
9997,5180458,firstly this much ballyhooed compliance annou...,0.0
9998,6a28b78723c1c6f2,you are so fucking dummm man i just dnt get it...,1.0


In [8]:
toxic_frame.isnull().sum()

id              0
comment_text    0
target          0
dtype: int64

In [13]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
toxic_frame['comment_text']= toxic_frame['comment_text'].apply(lambda x:expand_contractions(x));

In [14]:

toxic_frame['comment_text'] = toxic_frame['comment_text'].apply(
    lambda x: re.sub(r'\w*\d\w*', '', x)
)
toxic_frame['comment_text']=toxic_frame['comment_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
# Removing extra spaces
toxic_frame['comment_text']= toxic_frame['comment_text'].apply(lambda x: re.sub(' +',' ',x))
#turning all to lower case
toxic_frame['comment_text']= toxic_frame['comment_text'].apply(lambda x: x.lower())
for index,text in enumerate(toxic_frame['comment_text'][35:40]):
  print('Review %d:\n'%(index+1),text)

Review 1:
 and gas bag and liar
Review 2:
 this ip has again been temporarily blocked for reason of vandalism if you will write and edit responsibly in the future you may contribute here after the block expires continued vandalism can result in a longer term block mar coordinated universal time
Review 3:
 enjoy it while it lasts the islamification of canada has begun
Review 4:
 the people of alaska deserve less money for programs and services that benefit them rep mike hawker apparently what an idiot
Review 5:
 nixon was just going to keep firing people until he got what many scholars believe he had the legal right to do legal and moral being distinct issues here why gut the doj with another dozen firings and leave things in disarray bork appointed the subsequent prosecutor who drove nixon out of office


In [23]:
#removing stop words
stop_words = set(stopwords.words('english'))
toxic_frame['comment_text']= toxic_frame['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [19]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized)

# Apply to the column
toxic_frame['comment_text'] = toxic_frame['comment_text'].apply(lemmatize_text);


In [20]:
toxic_frame = toxic_frame.dropna()

toxic_frame['comment_text'] = toxic_frame['comment_text'].astype('string')


In [21]:
toxic_frame.isnull().sum()

id              0
comment_text    0
target          0
dtype: int64

In [22]:
toxic_frame['comment_text'].dtype


string[python]

In [23]:
train_set_len = int(len(toxic_frame)*0.80);

test_set_len = int(len(toxic_frame)*0.10);

val_set_len = int(len(toxic_frame)*0.10);

toxic_frame.loc[:train_set_len,'split'] = "train";

toxic_frame.loc[train_set_len:train_set_len + test_set_len,'split'] = "test";

toxic_frame.loc[train_set_len + test_set_len:,'split'] = "val";

toxic_frame.to_csv(f"../data/data_split.csv");

args.data_split = "../data/data_split.csv";

In [31]:
def pad_sequence(tokenizer,encoded_ids, max_length, pad_id = args.pad_token):
    
    # Truncate if too long
    if len(encoded_ids) > max_length:
        encoded_ids = encoded_ids[:max_length]
    
    # Add padding
    padding = [pad_id] * (max_length - len(encoded_ids))
    return encoded_ids + padding;

In [25]:
class Vectorizer():
    
    def __init__(self,tokenizer,vocabulary):
        self.tokenizer = tokenizer;
        self.vocabulary = vocabulary;
        pass;
    
    @classmethod
    def from_dataframe(cls, csvFile,columns):
        """
        Instantiate the vectorizer from the dataset dataframe
        
        Args:

        dataFrame (pandas.DataFrame): Dataset 
        xColumnName : name of features column.

        Returns: an instance of the Vectorizer

        """

        datafile = pd.read_csv(csvFile);

        tokens_iterator = [];

        tokenizer = torchtext.data.utils.get_tokenizer("basic_english");
        
        for i,row in datafile[[columns[0]]].iterrows():
            text = row[columns[0]];
            # print(type(text));
            text = str(text)
            tokens = tokenizer(text);
            tokens_iterator.append(tokens);

        caption_vocab = tv.build_vocab_from_iterator(tokens_iterator,min_freq = 3,specials=[args.pad_token, args.unk_token, args.start_token , args.end_token]);
        
        caption_vocab.set_default_index(caption_vocab[args.unk_token]);
   
        return cls(tokenizer,caption_vocab);
    
    
    
    def vectorize(self,sentence , max_length , add_special = False):
        
        """
        Create a numerical vector for the sentence
        
        Args:
        tokenizer: sentencepiece tokenizer.
        sentence (str): the sentence
        add_special (boolean) : Adding special tokens (<sos>,<eos>).
        
        Returns:
        Tokens indices vector.
        
        """
        vector = self.tokenizer(sentence);
        vector = pad_sequence(self.tokenizer,vector, max_length);
        ids = [];
        for i,token in enumerate(vector):
            ids.append(self.vocabulary[token]);
        
        return ids;


In [ ]:
class CustomDataSet(Dataset):
    
    def __init__(self,datafile,vectorizer):
        
        dataFrame = pd.read_csv(datafile);
        
        self._dataFrame = dataFrame;
        self._vectorizer = vectorizer;
        
        self.train_df = self._dataFrame[self._dataFrame.split == "train"];
        self.train_size = len(self.train_df);
        
        self.val_df = self._dataFrame[self._dataFrame.split == "val"];
        self.val_size = len(self.val_df);
        
        self.test_df = self._dataFrame[self._dataFrame.split == "test"];
        self.test_size = len(self.test_df);

        self._lookup_dict = {'train': (self.train_df, self.train_size),
        'val': (self.val_df, self.val_size),
        'test': (self.test_df, self.test_size)};
        
        self.set_split();  # setting train as default data set.

    
    @classmethod
    def load_dataset_and_make_vectorizer(cls,datafile):
        datafile = datafile;
        return cls(datafile, Vectorizer.from_dataframe(datafile,['comment_text']));

    
    def get_vectorizer(self):
        return self._vectorizer;
    
    
    def set_split(self, split="train"):
        
        """ selects the splits in the dataset using a column in the dataframe
        Args:finaldata.csv
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split;

        self._target_df, self._target_size = self._lookup_dict[split];


    def __getitem__(self, index):
        row = self._target_df.iloc[index]
    
        text = row['comment_text']
        if not isinstance(text, str):
            text = str(text)          #String conversion to handle the float values

        source = self._vectorizer.vectorize(text, args.max_sentence_length)
        target = row['target']

        return torch.tensor(source), torch.tensor(target)

    
    def __len__(self):
        return self._target_size;


    

In [27]:
dataset = CustomDataSet.load_dataset_and_make_vectorizer(args.data_split)
dataloader = DataLoader(dataset,batch_size=20);


In [28]:
toxic_frame['comment_text'].dtype


string[python]

In [29]:
for x,y in dataloader:
    break;
print(x,y)

tensor([[223,  13,   5,  ...,   0,   0,   0],
        [ 48,   7, 345,  ...,   0,   0,   0],
        [ 32, 140,  48,  ...,   0,   0,   0],
        ...,
        [ 38,  17,   1,  ...,   0,   0,   0],
        [  1,  26,   5,  ...,   0,   0,   0],
        [  7,  14,  50,  ...,   0,   0,   0]]) tensor([0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
        0., 0.])


In [32]:

def prepare_embedding_matrix(dataset_words, pre_embeddings):
    word_to_idx = pre_embeddings.stoi
    embedding_matrix = pre_embeddings.vectors
    embedding_dim = embedding_matrix.shape[1]

    final_embeddings = torch.zeros((len(dataset_words), embedding_dim))

    for i, word in enumerate(dataset_words):
        if word in word_to_idx and word_to_idx[word] < embedding_matrix.shape[0]:
            final_embeddings[i] = embedding_matrix[word_to_idx[word]]
        else:
            embedding_i = torch.empty(embedding_dim)
            torch.nn.init.xavier_uniform_(embedding_i.unsqueeze(0))
            final_embeddings[i] = embedding_i

    embedding_layer = nn.Embedding(
        num_embeddings=len(dataset_words),
        embedding_dim=embedding_dim
    )
    embedding_layer.weight.data = final_embeddings

    return embedding_layer


In [33]:
embedding_layer = prepare_embedding_matrix(list(dataset.get_vectorizer().vocabulary.get_stoi().keys()),global_vectors);

In [ ]:

class LightDNN(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=128, output_dim=1, dropout=0.4):
        super(LightDNN, self).__init__()

        self.embedding = embedding_layer  # Use pretrained embedding layer
        self.embedding.weight.requires_grad = False  # Optional: freeze embeddings

        self.fc1 = nn.Linear(args.max_sentence_length * args.embedding_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)  # shape: (batch_size, seq_len, emb_dim)
        flat = embedded.view(embedded.size(0), -1)  # flatten to (batch_size, seq_len * emb_dim)
        x = F.relu(self.fc1(flat))
        x = self.dropout(x)
        return torch.sigmoid(self.fc2(x)).squeeze(1)  # shape: (batch_size,)


In [57]:
model = LightDNN(embedding_layer).to(args.device)


In [ ]:

criterion = nn.BCELoss()  # Binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)



In [60]:
EPOCHS = 15

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(args.device), targets.to(args.device).float()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")


Epoch 1 | Loss: 47.6308
Epoch 2 | Loss: 28.0304
Epoch 3 | Loss: 26.7108
Epoch 4 | Loss: 25.0589
Epoch 5 | Loss: 23.2185
Epoch 6 | Loss: 22.2950
Epoch 7 | Loss: 21.6566
Epoch 8 | Loss: 19.7100
Epoch 9 | Loss: 18.4953
Epoch 10 | Loss: 18.8449
Epoch 11 | Loss: 18.2155
Epoch 12 | Loss: 17.0898
Epoch 13 | Loss: 18.3261
Epoch 14 | Loss: 16.5155
Epoch 15 | Loss: 17.4979


In [63]:
#To save model training progress
checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item()
}
torch.save(checkpoint, 'checkpoint.pth')


In [ ]:
#To load the saved model
checkpoint = torch.load('checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()


LightDNN(
  (embedding): Embedding(9729, 300)
  (fc1): Linear(in_features=30000, out_features=128, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
dataset.set_split("test")
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, targets in DataLoader(dataset, batch_size=32):
        inputs = inputs.to(args.device)
        targets = targets.to(args.device).float()

        outputs = model(inputs)
        preds = (outputs > 0.5).float()

        all_preds.extend(preds.cpu().tolist())
        all_targets.extend(targets.cpu().tolist())

# Compute metrics
acc = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds)
recall = recall_score(all_targets, all_preds)
f1 = f1_score(all_targets, all_preds)

print(f"Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Test Accuracy: 0.8939, Precision: 0.8510, Recall: 0.9537, F1 Score: 0.8994
